In [1]:
import pathlib

import pandas as pd
import toml

In [2]:
output_path = pathlib.Path("../screenA/idr0000-screenA-processed.txt")

In [3]:
# path to the data (processed)
PBMC_path = pathlib.Path(
    "../../data/PBMC_preprocess_sc_norm_no_fs_aggregated_nomic.parquet"
)
SHSY5Y_path = pathlib.Path(
    "../../data/SHSY5Y_preprocess_sc_norm_no_fs_aggregated_nomic.parquet"
)
ground_truth_path = pathlib.Path(
    "../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml"
)
# read the data
PBMC = pd.read_parquet(PBMC_path)
# read the ground truth
ground_truth = toml.load(ground_truth_path)
apoptosis = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control = ground_truth["Healthy"]["healthy_groups_list"]
# # make a column for the ground truth
PBMC
PBMC["ground_truth"] = "other"
PBMC["apoptosis"] = PBMC["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(apoptosis)
PBMC["pyroptosis"] = PBMC["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(
    pyroptosis
)
PBMC["control"] = PBMC["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(control)

# merge the apoptosis, pyroptosis and control columns into the ground_truth column
PBMC.loc[PBMC["apoptosis"], "ground_truth"] = "apoptosis"
PBMC.loc[PBMC["pyroptosis"], "ground_truth"] = "pyroptosis"
PBMC.loc[PBMC["control"], "ground_truth"] = "control"

# drop the apoptosis, pyroptosis and control columns
PBMC = PBMC.drop(
    columns=[
        "apoptosis",
        "pyroptosis",
        "control",
        "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
    ]
)


SHSY5Y = pd.read_parquet(SHSY5Y_path)
# read the ground truth
ground_truth = toml.load(ground_truth_path)

apoptosis = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control = ground_truth["Healthy"]["healthy_groups_list"]
# make a column for the ground truth
SHSY5Y["ground_truth"] = "other"
SHSY5Y["apoptosis"] = SHSY5Y["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(
    apoptosis
)
SHSY5Y["pyroptosis"] = SHSY5Y["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(
    pyroptosis
)
SHSY5Y["control"] = SHSY5Y["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(control)

# merge the apoptosis, pyroptosis and control columns into the ground_truth column
SHSY5Y.loc[SHSY5Y["apoptosis"], "ground_truth"] = "apoptosis"
SHSY5Y.loc[SHSY5Y["pyroptosis"], "ground_truth"] = "pyroptosis"
SHSY5Y.loc[SHSY5Y["control"], "ground_truth"] = "control"

# drop the apoptosis, pyroptosis and control columns
SHSY5Y = SHSY5Y.drop(
    columns=[
        "apoptosis",
        "pyroptosis",
        "control",
        "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
    ]
)

# concat the two dataframes
data = pd.concat([PBMC, SHSY5Y], axis=0)
data.head()
data.reset_index(drop=True, inplace=True)
data

,Metadata_Well,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,Cytoplasm_AreaShape_Compactness,...,VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU],oneb_Treatment_Dose_Inhibitor_Dose,ground_truth
0,B02,0.119709,0.129981,0.023535,0.012792,0.021824,0.011274,0.022585,0.012096,-0.032252,...,0.385081,1.000000,0.000000,0.430111,0.538503,0.784695,0.468448,0.237545,LPS_0.010_ug_per_ml_DMSO_0.025_%,pyroptosis
1,B03,0.095616,0.096278,0.010884,0.029580,0.009907,0.028735,0.010267,0.029245,-0.033178,...,0.327491,0.390866,0.406489,0.412096,0.104830,0.812933,0.518536,0.244397,LPS_0.010_ug_per_ml_DMSO_0.025_%,pyroptosis
2,B04,-0.031231,-0.046841,-0.042766,0.055657,-0.041048,0.057418,-0.042007,0.056635,-0.095278,...,0.357476,0.346884,0.477553,0.427658,0.642061,0.249380,0.627712,0.318350,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,pyroptosis
3,B05,-0.024048,-0.037242,-0.058885,0.039394,-0.057277,0.040976,-0.058201,0.040268,-0.081000,...,0.588899,0.828371,0.484102,0.294634,0.673648,0.236793,0.557634,0.350429,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,pyroptosis
4,B06,-0.038195,-0.035832,0.009600,0.017445,0.010187,0.018096,0.009889,0.017761,0.022371,...,0.395392,0.560129,0.504521,0.490444,0.258834,0.238358,0.524276,0.250670,DMSO_0.100_%_DMSO_0.025_%,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,O17,0.068040,0.129430,-0.005135,0.006749,-0.013720,-0.001266,-0.009666,0.003002,-0.053652,...,0.368463,1.000000,0.277555,0.315176,0.616905,0.560623,0.531130,0.511373,LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-...,control
297,O20,0.090042,0.184824,0.009052,-0.012419,-0.001640,-0.023019,0.003463,-0.018012,-0.008031,...,0.448746,0.195383,0.073424,0.288437,0.665233,0.547869,0.580180,0.470265,LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-...,control
298,O21,0.040557,0.106181,0.012091,0.078303,0.004443,0.070628,0.008757,0.074141,-0.039158,...,0.610956,0.528192,0.291855,0.267683,0.696918,0.608740,0.592028,0.499456,LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-...,control
299,O22,0.040488,0.031649,0.025968,0.033714,0.024841,0.029542,0.025255,0.031351,-0.078584,...,0.525615,0.539442,0.577948,0.138297,0.509006,0.663276,0.597001,0.403857,media_ctr_0.0_0_Media_ctr_0.0_0,control


In [4]:
# read in the nELISA data
nELISA_PBMC_path = pathlib.Path(
    "/home/lippincm/Documents/ML/Interstellar_Analysis/2.Nomic_nELISA_Analysis/Data/clean/Plate2/nELISA_plate_430420_PBMC_clean.parquet"
)
nELISA_SHSY5Y_path = pathlib.Path(
    "/home/lippincm/Documents/ML/Interstellar_Analysis/2.Nomic_nELISA_Analysis/Data/clean/Plate2/nELISA_plate_430420_SHSY5Y_clean.parquet"
)
nELISA_PBMC = pd.read_parquet(nELISA_PBMC_path)
nELISA_SHSY5Y = pd.read_parquet(nELISA_SHSY5Y_path)
nELISA_PBMC
# drop all all columns that are not position x or do not conatin [NSU]
nELISA_PBMC = nELISA_PBMC[
    nELISA_PBMC.columns[nELISA_PBMC.columns.str.contains("position|NSU")]
]
nELISA_SHSY5Y = nELISA_SHSY5Y[
    nELISA_SHSY5Y.columns[nELISA_SHSY5Y.columns.str.contains("position|NSU")]
]
nELISA_PBMC.drop(columns=["plate_position", "position_y"], inplace=True)
nELISA_SHSY5Y.drop(columns=["plate_position", "position_y"], inplace=True)
# merge the two dataframes
nELISA = pd.concat([nELISA_PBMC, nELISA_SHSY5Y], axis=0)
nELISA

,position_x,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,B06,0.097710,0.461685,0.270477,0.514695,0.479281,0.270494,0.708849,0.134432,0.350986,...,0.386063,0.469875,0.395392,0.560129,0.504521,0.490444,0.258834,0.238358,0.524276,0.250670
1,C06,0.064513,0.451181,0.246274,0.471026,0.269795,0.204498,0.247611,0.322087,0.350642,...,0.454460,0.570146,0.032391,0.476656,0.315426,0.589522,0.381170,0.168645,0.455092,0.228752
2,I06,0.061860,0.196318,0.236491,0.474891,0.174672,0.824721,0.704521,0.254823,0.443939,...,0.755683,0.374554,0.486915,0.389375,0.369421,0.680276,0.182956,0.263281,0.213596,0.064645
3,J06,0.060998,0.596601,0.129926,0.302610,0.559309,0.087533,0.541110,0.350256,0.528260,...,0.254542,0.630644,0.586271,0.258029,0.561051,0.551671,0.582053,0.087565,0.140992,0.234191
4,B07,0.061116,0.490832,0.339510,0.453362,0.414653,0.424223,0.702561,0.203464,0.502516,...,0.424098,0.493033,0.171562,0.615867,0.288153,0.506528,0.264141,0.296782,0.541689,0.167078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,M15,0.537553,0.454080,0.121235,0.537197,0.299965,0.656053,0.495266,0.404481,0.483184,...,0.490550,0.107408,0.071024,0.445528,0.601425,0.302434,0.000000,0.754374,0.676954,0.570847
150,B13,0.972764,0.738218,0.401194,0.722364,0.043239,0.412779,0.457284,0.683110,0.335358,...,0.493924,0.187623,0.147899,0.500155,0.064014,0.275746,0.385698,0.176153,0.727225,0.529847
151,I13,0.725803,0.560143,0.383278,0.320750,0.668227,0.520634,0.669621,0.314319,0.460774,...,0.141638,0.252980,0.437131,0.429447,0.569871,0.276792,0.518688,0.428738,0.442567,0.486011
152,O22,0.356083,0.349350,0.335341,0.790661,0.250488,0.487088,0.237995,0.878783,0.226819,...,0.472063,0.204027,0.525615,0.539442,0.577948,0.138297,0.509006,0.663276,0.597001,0.403857


In [5]:
# add the nELISA data to the data dataframe merge on the position_x column
data = pd.merge(
    data, nELISA, left_on="Metadata_Well", right_on="position_x", how="left"
)

In [6]:
print(PBMC.shape, SHSY5Y.shape)

(154, 1247) (154, 1253)


In [7]:
path = pathlib.Path("../screenA/idr0000-screenA-library.txt")
df = pd.read_csv(path, sep="\t")
df.head()
print(df.columns.to_list())
# keep only the columns we need
df = df[
    [
        "Well",
        "Characteristics[cell type]",
        "inhibitor",
        "inhibitor_concentration",
        "inhibitor_concentration_unit",
        "inducer1",
        "inducer1_concentration",
        "inducer1_concentration_unit",
        "inducer2",
        "inducer2_concentration",
        "inducer2_concentration_unit",
        "Plate",
    ]
]

['Well', 'Characteristics[cell type]', 'inhibitor', 'inhibitor_concentration', 'inhibitor_concentration_unit', 'inducer1', 'inducer1_concentration', 'inducer1_concentration_unit', 'inducer2', 'inducer2_concentration', 'inducer2_concentration_unit', 'Plate', 'Term Source 1 REF', 'Term Source 1 Accession', 'Channels', 'Compound1', 'Compound1_InChIKey', 'Compound1_SMILES', 'Compound1_PubChem_URL', 'Compound1_Control_type', 'Compound2', 'Compound2_InChIKey', 'Compound2_SMILES', 'Compound2_PubChem_URL', 'Compound2_Control_type', 'Compound3', 'Compound3_InChIKey', 'Compound3_SMILES', 'Compound3_PubChem_URL', 'Compound3_Control_type']


In [8]:
# add df and data
data = data.merge(df, left_on="Metadata_Well", right_on="Well")
# replace NA with ""
data.fillna("", inplace=True)
data

,Metadata_Well,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,...,inhibitor,inhibitor_concentration,inhibitor_concentration_unit,inducer1,inducer1_concentration,inducer1_concentration_unit,inducer2,inducer2_concentration,inducer2_concentration_unit,Plate
0,B02,-0.032252,0.060488,0.105783,-0.029521,0.005120,-0.052854,-0.039927,-0.004497,0.022885,...,DMSO,0.025,%,LPS,0.010,µg/ml,,,,plate_1
1,B03,-0.033178,0.054449,0.060468,-0.073702,-0.012549,-0.076747,-0.072953,0.003722,-0.001647,...,DMSO,0.025,%,LPS,0.010,µg/ml,,,,plate_1
2,B04,-0.095278,0.085873,-0.114237,-0.175646,-0.003159,-0.054675,-0.120368,0.084387,-0.047532,...,DMSO,0.025,%,LPS,100.000,µg/ml,Nigericin,1.0,µM,plate_1
3,B05,-0.081000,0.078124,-0.104225,-0.164719,0.001145,-0.070612,-0.131044,0.088143,-0.062731,...,DMSO,0.025,%,LPS,100.000,µg/ml,Nigericin,1.0,µM,plate_1
4,B06,0.022371,-0.022614,-0.039573,-0.049300,-0.013386,-0.035579,-0.026341,-0.034624,-0.015618,...,DMSO,0.025,%,DMSO,0.100,%,,,,plate_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,O19,-0.030737,0.013307,0.087081,0.062723,0.030207,0.011448,-0.006952,0.048732,0.005120,...,Media ctr,,,media ctr,,µg/ml,,,,plate_1
304,O20,-0.008031,-0.039525,0.227378,-0.005209,0.012483,-0.121338,-0.041426,-0.011089,0.034808,...,Z-VAD-FMK,100.0,µM,LPS,1.000,µg/ml,Nigericin,10.0,µM,plate_1
305,O21,-0.039158,-0.022291,0.151747,0.022673,0.001550,-0.070861,-0.024675,0.016000,0.026620,...,Z-VAD-FMK,100.0,µM,LPS,1.000,µg/ml,Nigericin,10.0,µM,plate_1
306,O22,-0.078584,0.058422,0.033983,0.037911,0.019646,0.063310,0.006814,0.056728,0.029180,...,Media ctr,,,media ctr,,,,,,plate_1


In [9]:
# write the data to a txt file
data.to_csv(output_path, sep="\t", index=False)